I used @radek1's approach for voting ensemble: https://www.kaggle.com/code/radek1/2-methods-how-to-ensemble-predictions

# Loading the data

In [9]:
!pip install polars

In [13]:
import polars as pl
paths = ['0.578_Ensemble of Public Notebooks_submission.csv', # 0.578
         'Tuning Candidate ReRank Model[LB 0.577]_submission.csv', # 0.577
         'easy-to-use co-visitation matrix function[LB 0.577]submission.csv', # 0.577
         'Fast Handcrafted model & Recall@20[LB 0.575]_submission.csv', #0.575
         'Otto Fast CPU End-to-End Pipeline_submission.csv'#0.574
        ]
#weights = [0.577, 0.577, 0.575, 0.533]
weights = [1,1,1,1,1]

In [14]:
def read_sub(path, weight=1): # by default let us assing the weight of 1 to predictions from each submission, this will be akin to a standard vote ensemble
    '''a helper function for loading and preprocessing submissions'''
    return (
        pl.read_csv(path)
            .with_column(pl.col('labels').str.split(by=' '))
            .with_column(pl.lit(weight).alias('vote'))
            .explode('labels')
            .rename({'labels': 'aid'})
            .with_column(pl.col('aid').cast(pl.UInt32)) # we are casting the `aids` to `Int32`! memory management is super important to ensure we don't run out of resources
            .with_column(pl.col('vote').cast(pl.UInt8))
    )

In [15]:
subs = [read_sub(path, weight=weights[i]) for (i, path) in enumerate(paths)]
subs[0].head()

C:\Users\adamo\AppData\Local\Temp\ipykernel_29840\3887510293.py:4: DeprecationWarning: `with_column` has been deprecated in favor of `with_columns`. This method will be removed in version 0.17.0
  pl.read_csv(path)


session_type,aid,vote
str,u32,u8
"""14279927_carts...",872695,1
"""14279927_carts...",922440,1
"""14279927_carts...",67054,1
"""14279927_carts...",153333,1
"""14279927_carts...",215472,1


In [16]:
subs = subs[0].join(subs[1], how='outer', on=['session_type', 'aid']).join(subs[2], how='outer', on=['session_type', 'aid'], suffix='_right2')
subs.head()

session_type,aid,vote,vote_right,vote_right2
str,u32,u8,u8,u8
"""12899779_click...",59625,1,1,1
"""12899779_click...",1253524,1,1,1
"""12899779_click...",737445,1,1,1
"""12899779_click...",438191,1,1,1
"""12899779_click...",731692,1,1,1


In [17]:
subs = (subs
    .fill_null(0)
    .with_column((pl.col('vote') + pl.col('vote_right') + pl.col('vote_right2')).alias('vote_sum'))
    .drop(['vote', 'vote_right', 'vote_right2'])
    .sort(by='vote_sum')
    .reverse()
)

subs.head()

C:\Users\adamo\AppData\Local\Temp\ipykernel_29840\3427031507.py:1: DeprecationWarning: `with_column` has been deprecated in favor of `with_columns`. This method will be removed in version 0.17.0
  subs = (subs


session_type,aid,vote_sum
str,u32,u8
"""14571581_carts...",196038,3
"""14571581_carts...",1497245,3
"""14571581_carts...",174670,3
"""14571581_carts...",1764910,3
"""14571581_carts...",984794,3


In [18]:
%%time
preds = subs.groupby('session_type').agg([
    pl.col('aid').head(20).alias('labels')
])

preds = preds.with_column(pl.col('labels').apply(lambda lst: ' '.join([str(aid) for aid in lst])))

<timed exec>:5: DeprecationWarning: `with_column` has been deprecated in favor of `with_columns`. This method will be removed in version 0.17.0


Wall time: 3min 13s


In [19]:
preds.write_csv('submission.csv')